In [11]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [12]:
#データの読み込み
#data1 = pd.read_csv("/home/mizuguchi/workspace/Lifelog-6/vaisl_gps.csv")
data = pd.read_csv("/home/mizuguchi/workspace/Lifelog-6/lsc22_metadata.csv")
data.head()
#data.size→19692000

#local_timeと心拍数や緯度経度の情報持っている行を取り出す
data = data.dropna(subset=["local_time", "latitude", "longitude", "heart_rate(bpm)", "ImageID"])
data.head()
#data.size→1518050

/tmp/ipykernel_346888/985038506.py:3: DtypeWarning: Columns (13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/home/mizuguchi/workspace/Lifelog-6/lsc22_metadata.csv")


,Unnamed: 0,minute_id,utc_time,local_time,latitude,longitude,altitude,semantic_name,time_zone,heart_rate(bpm),...,album name,sleep_level,awake,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_efficiency,ImageID
654,654,20190101_1054,2019-01-01 10:54:42,2019-01-01 10:54:42,53.390071,-6.145660,52.0,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,110.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_105421_000.jpg', '20190101_105453_0..."
659,659,20190101_1059,2019-01-01 10:59:44,2019-01-01 10:59:44,53.389955,-6.145689,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,96.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_105922_000.jpg', '20190101_105953_0..."
660,660,20190101_1100,2019-01-01 11:00:15,2019-01-01 11:00:15,53.389978,-6.145769,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,99.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_110026_000.jpg', '20190101_110058_0..."
661,661,20190101_1101,2019-01-01 11:01:16,2019-01-01 11:01:16,53.389986,-6.145771,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,99.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,['20190101_110130_000.jpg']
662,662,20190101_1102,2019-01-01 11:02:28,2019-01-01 11:02:28,53.389992,-6.145763,51.0,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,93.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,"['20190101_110202_000.jpg', '20190101_110234_0..."


In [13]:
import pandas as pd
import ast

# ImageID列を文字列からリストに変換
data["ImageID"] = data["ImageID"].apply(ast.literal_eval)
#data['ImageID'] = data['ImageID'].str.split(',')  # ','で分割→これでもいけるかも

# 各画像ごとに行を展開（explode）
data = data.explode("ImageID").reset_index(drop=True)
data.head()

,Unnamed: 0,minute_id,utc_time,local_time,latitude,longitude,altitude,semantic_name,time_zone,heart_rate(bpm),...,album name,sleep_level,awake,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,sleep_efficiency,ImageID
0,654,20190101_1054,2019-01-01 10:54:42,2019-01-01 10:54:42,53.390071,-6.145660,52.0,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,110.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_105421_000.jpg
1,654,20190101_1054,2019-01-01 10:54:42,2019-01-01 10:54:42,53.390071,-6.145660,52.0,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,110.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_105453_000.jpg
2,659,20190101_1059,2019-01-01 10:59:44,2019-01-01 10:59:44,53.389955,-6.145689,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,96.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_105922_000.jpg
3,659,20190101_1059,2019-01-01 10:59:44,2019-01-01 10:59:44,53.389955,-6.145689,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,96.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_105953_000.jpg
4,660,20190101_1100,2019-01-01 11:00:15,2019-01-01 11:00:15,53.389978,-6.145769,NaN,"72 Verbena Avenue, Dublin, County Dublin, D13 ...",Europe/Dublin,99.0,...,NaN,NaN,NaN,0.0,299.0,52.0,5.0,351.0,90.0,20190101_110026_000.jpg


In [14]:
# ファイル存在チェック関数
def image_exists(row, image_root):
    image_id = row["ImageID"]
    year_month = image_id[:6]
    day = image_id[6:8]
    img_path = os.path.join(image_root, year_month, day, image_id)
    return os.path.exists(img_path)

# 存在するかどうか確認
image_root = "/data/Lifelog-6/images"
data["image_exists"] = data.apply(image_exists, axis=1, image_root=image_root)

# 存在する画像のみ残す
data = data[data["image_exists"]].reset_index(drop=True)
data.drop(columns=["image_exists"], inplace=True)

print(f"画像が存在する行数: {len(data)}")

画像が存在する行数: 131100


In [15]:
#local_time列に対して、整形
data["local_time"] = pd.to_datetime(data["local_time"])
#1日の経過時間を算出し、time_seconds列に代入
data["time_seconds"] = (data["local_time"].dt.hour * 3600) + (data["local_time"].dt.minute * 60) + (data["local_time"].dt.second)
#必要なデータ以外をドロップ
data = data.dropna(subset=["time_seconds", "latitude", "longitude", "heart_rate(bpm)", "ImageID"])

In [16]:
#データセットの作成
class MultimodalDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # 心拍と緯度経度
        hr = torch.tensor([row["heart_rate(bpm)"]], dtype=torch.float32)
        latlon = torch.tensor([row["latitude"], row["longitude"]], dtype=torch.float32)
        
        # 画像読み込み
        month = row["ImageID"][:6]
        day = row["ImageID"][6:8]

        img_path = os.path.join(self.image_dir, month, day, row["ImageID"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        # 正解：記録時刻（秒）
        label = torch.tensor(row["time_seconds"], dtype=torch.float32)

        return image, hr, latlon, label

In [17]:
#画像前処理
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [18]:
#DataLoaderの作成
#1.検証用データの必要性を確認
#2.画像のディレクトリ設定→特にフォルダから取り出せるように記述する必要あり
#3.image_transformは適切か確認する
#4.バッチサイズは32で適切か

#データを学習データとテストデータに分割
train_data, test_data = train_test_split(data, test_size=0.2)

#データセットを作成
train_dataset = MultimodalDataset(train_data, image_dir="/home/mizuguchi/workspace/transfer2-main/DMR/Lifelog-6/images", transform=image_transform)
test_dataset = MultimodalDataset(test_data, image_dir="/home/mizuguchi/workspace/transfer2-main/DMR/Lifelog-6/images", transform=image_transform)

#データローダーの作成
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
#時間推定モデルの構築
#1.画像の特徴抽出方法はResnet18で良いのか
#2.画像に対して128次元のベクトル表現で適切なのか
class time_estimate(nn.Module):
    def __init__(self):
        super().__init__()
        #画像データから特徴量抽出
        self.cnn = models.resnet18(pretrained=True)
        # self.cnn_places = ~
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 128)
        #センサデータから特徴量抽出
        self.sensor = nn.Sequential(
            nn.Linear(3, 32),
            nn.ReLU(),
            nn.Linear(32, 64)
        )
        #画像データとセンサデータと特徴量の結合
        self.concat = nn.Sequential(
            nn.Linear(64+128, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
        #次は3で出力
    
    def forward(self, img, hr, latlon):
        img_feat = self.cnn(img)
        sensor_feat = self.sensor(torch.cat([hr, latlon], dim=1))
        concat_feat = torch.concat([img_feat, sensor_feat], dim=1)
        output = self.concat(concat_feat)
        return output

In [20]:
#時間推定モデルの学習
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = time_estimate().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for img, sensor, latlon, label in train_loader:
        img, sensor, latlon, label = img.to(device), sensor.to(device), latlon.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(img, sensor, latlon)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch:{epoch+1} loss:{total_loss / len(train_loader)}")

/home/mizuguchi/miniconda3/envs/lifelog/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mizuguchi/miniconda3/envs/lifelog/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/mizuguchi/miniconda3/envs/lifelog/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  retu

Epoch:0 loss:350531782.6723612
Epoch:1 loss:276561130.9042099
Epoch:2 loss:276431006.29164124
Epoch:3 loss:276742340.51982915
Epoch:4 loss:276522693.58877367
Epoch:5 loss:276415465.3471629
Epoch:6 loss:276277260.9200732
Epoch:7 loss:276457805.7742526
Epoch:8 loss:276282494.89200735
Epoch:9 loss:276300028.9395973


In [21]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
all_pred = []
all_label = []
with torch.no_grad():
    for image, hr, latlon, label in test_loader:
        image, hr, latlon, label = image.to(device), hr.to(device), latlon.to(device), label.to(device)
        output = model(image, hr, latlon)
        all_pred.append(output)
        all_label.append(label)

mse = mean_squared_error(all_pred, all_label)    
print(f"mse:{mse}")

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for img, hr, latlon, label in test_loader:
        img, hr, latlon, label = img.to(device), hr.to(device), latlon.to(device), label.to(device)
        output = model(img, hr, latlon)
        all_preds.extend(output.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

mse = mean_squared_error(all_labels, all_preds)
mae = mean_absolute_error(all_labels, all_preds)
print(f"MSE: {mse:.2f}, MAE: {mae:.2f}秒")


MSE: 271988128.00, MAE: 14291.77秒


In [23]:
def seconds_to_time_str(seconds):
    seconds = int(seconds) % 86400  # 24時間制に丸める
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    return f"{hours:02}:{minutes:02}"


In [30]:
# 任意の1件のデータで確認
sample = test_dataset[7000]
img, hr, latlon, label = sample
model.eval()
with torch.no_grad():
    pred = model(img.unsqueeze(0).to(device),
                 hr.unsqueeze(0).to(device),
                 latlon.unsqueeze(0).to(device))
    pred_time = pred.item()
    true_time = label.item()

print("予測：", seconds_to_time_str(pred_time))
print("正解：", seconds_to_time_str(true_time))


予測： 14:37
正解： 12:36
